<a href="https://colab.research.google.com/github/atanuGuin/NLP_Coursework/blob/master/Enron_Email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Necessary Libraries

In [ ]:
import os, email
import pandas as pd
import glob
import matplotlib.pyplot as plt
import re
import string
import time
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Define the data path

In [ ]:
PATH = '/content/drive/MyDrive/Demo/enron_with_categories'


#Loading the Data based on 6 categories

In [ ]:
category_dict = {1: "Company Business, Strategy, etc." ,
                 2: "Purely Personal",
                 3: "Personal but in professional context",
                 4: "Logistic Arrangements",
                 5: "Employment arrangements",
                 6: "Document editing/checking"}
                 
files = []
text = []
category = []


for path in tqdm(glob.glob(PATH + "/[1-6]/*.txt")):
    batch,filename = path.split("/")[-2:]
    contents = open(path, "r").read()
    files.append(filename)
    text.append(contents)
    category.append(category_dict[int(batch)])



  0%|          | 0/1663 [00:00<?, ?it/s]

In [ ]:
emails_df = pd.DataFrame(list(zip(files,text, category)),columns = ["file", "message", "category"])

In [ ]:
emails_df.to_csv("emails.csv", index=False)

In [ ]:
emails_df = pd.read_csv("emails.csv" )

In [ ]:
emails_df

,file,message,category
0,175741.txt,Message-ID: <31425995.1075847622214.JavaMail.e...,"Company Business, Strategy, etc."
1,55205.txt,Message-ID: <18699857.1075856630781.JavaMail.e...,"Company Business, Strategy, etc."
2,120674.txt,Message-ID: <20878896.1075843391140.JavaMail.e...,"Company Business, Strategy, etc."
3,173929.txt,Message-ID: <10985446.1075846163264.JavaMail.e...,"Company Business, Strategy, etc."
4,173918.txt,Message-ID: <966711.1075846162973.JavaMail.eva...,"Company Business, Strategy, etc."
...,...,...,...
1658,229326.txt,Message-ID: <26808496.1075853126387.JavaMail.e...,Personal but in professional context
1659,12176.txt,Message-ID: <31386690.1075860837352.JavaMail.e...,Personal but in professional context
1660,114495.txt,Message-ID: <24509319.1075842948701.JavaMail.e...,Personal but in professional context
1661,12174.txt,Message-ID: <28168211.1075860837271.JavaMail.e...,Personal but in professional context


#Preprocessing the Data

##Step 1: Extract different parts an email from the text files like Sender, Receiver, Subject, Content, Message type etc. to create a dataframe

In [ ]:
## Helper functions
def get_text_from_email(msg, max_word_len=30):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            payload = part.get_payload()
            payload = ' '.join(filter(lambda x: len(x) < max_word_len,  payload.split()))
            parts.append( payload )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

In [ ]:
# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))

# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
# emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])

# cleanup
del messages
emails_df.drop('message', axis=1, inplace=True)

emails_df.head()

,file,category,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content
0,175741.txt,"Company Business, Strategy, etc.",<31425995.1075847622214.JavaMail.evans@thyme>,"Fri, 9 Mar 2001 09:28:00 -0800 (PST)",(steven.kean@enron.com),(pamela.lesh@enron.com),Re: Several matters,"joe.hartsoe@enron.com, james.derrick@enron.com...",1.0,text/plain; charset=us-ascii,7bit,"joe.hartsoe@enron.com, james.derrick@enron.com...",Steven J Kean,Pamela Lesh,"Joe Hartsoe, James Derrick, Linda Robertson",,\Steven_Kean_June2001_1\Notes Folders\All docu...,KEAN-S,skean.nsf,Thanks for the message. It's good to hear from...
1,55205.txt,"Company Business, Strategy, etc.",<18699857.1075856630781.JavaMail.evans@thyme>,"Thu, 1 Mar 2001 06:29:00 -0800 (PST)",(hillh@stanford.edu),(vkamins@enron.com),possible RTP conference,None,1.0,text/plain; charset=us-ascii,7bit,None,Hill Huntington <hillh@stanford.edu>,vkamins@enron.com,,,\Vincent_Kaminski_Jun2001_5\Notes Folders\Stan...,Kaminski-V,vkamins.nsf,Dear Mr. Kaminski: Thank you for talking with ...
2,120674.txt,"Company Business, Strategy, etc.",<20878896.1075843391140.JavaMail.evans@thyme>,"Wed, 14 Mar 2001 05:47:00 -0800 (PST)",(alan.comnes@enron.com),(susan.mara@enron.com),Re: You Can Put A Stake in Their Hearts but Th...,"richard.shapiro@enron.com, james.steffes@enron...",1.0,text/plain; charset=ANSI_X3.4-1968,quoted-printable,"richard.shapiro@enron.com, james.steffes@enron...",Alan Comnes,Susan J Mara,"Richard Shapiro, James D Steffes, Joe Hartsoe,...",,\Jeff_Dasovich_June2001\Notes Folders\All docu...,DASOVICH-J,jdasovic.nsf,Here's the proposal to form the Team. Note tha...
3,173929.txt,"Company Business, Strategy, etc.",<10985446.1075846163264.JavaMail.evans@thyme>,"Wed, 2 Aug 2000 11:33:00 -0700 (PDT)",(steven.kean@enron.com),(scott.tholan@enron.com),Enron projects with OPIC financing,None,1.0,text/plain; charset=us-ascii,7bit,None,Steven J Kean,Scott Tholan,,,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,OPIC information ---------------------- Forwar...
4,173918.txt,"Company Business, Strategy, etc.",<966711.1075846162973.JavaMail.evans@thyme>,"Wed, 2 Aug 2000 08:05:00 -0700 (PDT)",(steven.kean@enron.com),(jon.thomsen@enron.com),Ken Lay's Meeting with Fox,"ken.rice@enron.com, ricardo.charvel@enron.com,...",1.0,text/plain; charset=us-ascii,7bit,"ken.rice@enron.com, ricardo.charvel@enron.com,...",Steven J Kean,Jon Thomsen,"Ken Rice, Ricardo Charvel, Rosalee Fleming",,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,Jon -- Ricardo Charvel is pulling together the...


##Step 2: Checking the dataframe for unique values and dropping irrelevant features for email classification task


In [ ]:
print('shape of the dataframe:', emails_df.shape)
# Find number of unique values in each columns
for col in emails_df.columns:
    print(col, emails_df[col].nunique())

shape of the dataframe: (1663, 20)
file 1663
category 6
Message-ID 1663
Date 1571
From 172
To 696
Subject 1242
Cc 288
Mime-Version 1
Content-Type 2
Content-Transfer-Encoding 2
Bcc 288
X-From 202
X-To 842
X-cc 319
X-bcc 1
X-Folder 180
X-Origin 76
X-FileName 93
content 1554


## Step 3: Only Subject and Email content have been taken for the classification task

In [ ]:
emails_df.columns

Index(['file', 'category', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Cc',
       'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'Bcc',
       'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName',
       'content'],
      dtype='object')

In [ ]:
emails_df['text'] = emails_df['Subject'] + " " + emails_df['content']

In [ ]:
# drop the columns 'subject' and 'body'
emails_df.drop(['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Cc',
       'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'Bcc',
       'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'content'], axis=1, inplace=True)

In [ ]:
emails_df

,category,text
0,"Company Business, Strategy, etc.",Re: Several matters Thanks for the message. It...
1,"Company Business, Strategy, etc.",possible RTP conference Dear Mr. Kaminski: Tha...
2,"Company Business, Strategy, etc.",Re: You Can Put A Stake in Their Hearts but Th...
3,"Company Business, Strategy, etc.",Enron projects with OPIC financing OPIC inform...
4,"Company Business, Strategy, etc.",Ken Lay's Meeting with Fox Jon -- Ricardo Char...
...,...,...
1658,Personal but in professional context,RE: Confidential Concern Valeria is looking in...
1659,Personal but in professional context,A Supportive Note Dear Mr. Lay: I'm writing th...
1660,Personal but in professional context,[Second Delivery: WPTF Friday Amen Burrito] So...
1661,Personal but in professional context,Thank you all very much for your support - NOT...


##Step 4: Preprocessing the context by converting it to lowercase, removing extra lines and white spaces, removing punctuations and stopwords

In [ ]:
def preprocess(x):
    # lowercasing all the words
    x = x.lower()
    
    # remove extra new lines
    x = re.sub(r'\n+', ' ', x)
    
    # removing (replacing with empty spaces actually) all the punctuations
    x = re.sub("["+string.punctuation+"]", " ", x)
    
    # remove extra white spaces
    x = re.sub(r'\s+', ' ', x)
    
    return x

In [ ]:
start = time.time()
emails_df.loc[:,'text'] = emails_df.loc[:, 'text'].map(preprocess)

# remove stopwords
emails_df.loc[:, 'text'] = emails_df.loc[:, 'text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
end = time.time()
print("Execution time (sec): ",(end - start))

Execution time (sec):  5.656864881515503


In [ ]:
emails_df.head(10)

,category,text
0,"Company Business, Strategy, etc.",several matters thanks message good hear alrea...
1,"Company Business, Strategy, etc.",possible rtp conference dear mr kaminski thank...
2,"Company Business, Strategy, etc.",put stake hearts never die proposal form team ...
3,"Company Business, Strategy, etc.",enron projects opic financing opic information...
4,"Company Business, Strategy, etc.",ken lay meeting fox jon ricardo charvel pullin...
5,"Company Business, Strategy, etc.",thanks good article jeff dasovich enron com an...
6,"Company Business, Strategy, etc.",enron advertising mechanism survey employees s...
7,"Company Business, Strategy, etc.",enron advertising let ask jeff hines opinion r...
8,"Company Business, Strategy, etc.",tax valuations yr2000 fyi forwarded steven j k...
9,"Company Business, Strategy, etc.",arbitration reliance ongc aware forwarded stev...


##Step 5: Encoding the 6 labels for as target

In [ ]:
def label_encoder(data):
    class_le = LabelEncoder()
    # apply label encoder on the 'X-Folder' column
    y = class_le.fit_transform(data)
    return y

In [ ]:
y = label_encoder(emails_df['category'])
input_data = emails_df['text']

In [ ]:
np.unique(y)

array([0, 1, 2, 3, 4, 5])

In [ ]:
input_data.head()

0    several matters thanks message good hear alrea...
1    possible rtp conference dear mr kaminski thank...
2    put stake hearts never die proposal form team ...
3    enron projects opic financing opic information...
4    ken lay meeting fox jon ricardo charvel pullin...
Name: text, dtype: object

#Create TF-IDF Vectorizer For convering Words to vectors 

In [ ]:
start = time.time()
vectorizer = TfidfVectorizer(min_df=5, max_features=200)
X = vectorizer.fit_transform(input_data)

X = X.toarray()
print("X.shape: ",X.shape)

end = time.time()
print("Execution time (sec): ",(end - start))

X.shape:  (1663, 200)
Execution time (sec):  0.9461612701416016


#Creating a Dataframe to store the metrics

In [ ]:
# create dataframe to store results
metrics_data = {
    'Algorithm': ['Logistic', 'Gaussian NB', 'Multinomial NB','KNN', 'Decision Tree','SVM', 'RandomForestClassifier', 'AdaBoost', 'XGBoost','LGBMClassifier', 'ANN'],
    'Accuracy': '',
    'F1-Score': '',
    'time': ''
}
metrics_df = pd.DataFrame(metrics_data)
metrics_df

,Algorithm,Accuracy,F1-Score,time
0,Logistic,,,
1,Gaussian NB,,,
2,Multinomial NB,,,
3,KNN,,,
4,Decision Tree,,,
5,SVM,,,
6,RandomForestClassifier,,,
7,AdaBoost,,,
8,XGBoost,,,
9,LGBMClassifier,,,


#Instanciating the Models and run them one by one using 80% Train and 20% Test using Cross-Validation

In [ ]:
models = [LogisticRegression(), GaussianNB(), MultinomialNB(), KNeighborsClassifier(), DecisionTreeClassifier(), LinearSVC(), 
          RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42),
          AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=50),
          XGBClassifier(n_estimators=50),
          LGBMClassifier(n_estimators=50),
         MLPClassifier(hidden_layer_sizes=(5,))]

names = ["Logistic", "Gaussian NB","Multinomial NB", "KNN", "Decision Tree", "SVM", "RandomForestClassifier", "AdaBoost", "XGBoost", "LGBMClassifier", "ANN"]

acc_scores = []
f1_scores = []
exec_times = []

for model, name in zip(models, names):
    print(name)
    start = time.time()
    scoring = {
        'acc': 'accuracy',
        'f1_mac': 'f1_macro'
    }
    scores = cross_validate(model, X, y, cv=5, n_jobs=5, scoring=scoring)
    training_time = (time.time() - start)
    print("accuracy: ", scores['test_acc'].mean())
    print("f1_score: ", scores['test_f1_mac'].mean())
    print("time (sec): ", training_time)
    print("\n")
    
    acc_scores.append(scores['test_acc'].mean())
    f1_scores.append(scores['test_f1_mac'].mean())
    exec_times.append(training_time)
    


Logistic
accuracy:  0.5657838561453019
f1_score:  0.2113720657676954
time (sec):  0.7812583446502686


Gaussian NB
accuracy:  0.3000651253663302
f1_score:  0.18658048040717765
time (sec):  0.15970873832702637


Multinomial NB
accuracy:  0.5664459640363255
f1_score:  0.20233945669413486
time (sec):  0.14433598518371582


KNN
accuracy:  0.46055392742139734
f1_score:  0.21244174404100002
time (sec):  0.23575043678283691


Decision Tree
accuracy:  0.4852400593364449
f1_score:  0.2807722520645613
time (sec):  0.4298222064971924


SVM
accuracy:  0.5507507507507508
f1_score:  0.2574951910120188
time (sec):  0.28507518768310547


RandomForestClassifier
accuracy:  0.5687814320344441
f1_score:  0.2760438446093151
time (sec):  2.1151740550994873


AdaBoost
accuracy:  0.5399236585983573
f1_score:  0.2685060135390006
time (sec):  10.424881935119629


XGBoost
accuracy:  0.5555591736314627
f1_score:  0.2376068743278658
time (sec):  9.050057411193848


LGBMClassifier
accuracy:  0.5742013097434784
f1_s

In [ ]:
metrics_df['Accuracy'] = acc_scores
metrics_df['F1-Score'] = f1_scores
metrics_df['time'] = exec_times
metrics_df


,Algorithm,Accuracy,F1-Score,time
0,Logistic,0.565784,0.211372,0.781258
1,Gaussian NB,0.300065,0.186580,0.159709
2,Multinomial NB,0.566446,0.202339,0.144336
3,KNN,0.460554,0.212442,0.235750
4,Decision Tree,0.485240,0.280772,0.429822
5,SVM,0.550751,0.257495,0.285075
6,RandomForestClassifier,0.568781,0.276044,2.115174
7,AdaBoost,0.539924,0.268506,10.424882
8,XGBoost,0.555559,0.237607,9.050057
9,LGBMClassifier,0.574201,0.294554,11.472315


#Result Analysis: As we can see multiple models are giving us accuracy in the range on 50-60%. But if we consider F1 Score, we can say Decision Tree and Light GBM Classifier are doing better than other models. Since it is a imbalanced dataset where the email counts across different categories are not in the same range. So the accuracy is getting affected much. 